In [18]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import jaccard_score
from sklearn.metrics import make_scorer
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.ensemble import MajorityVotingClassifier
from skmultilearn.cluster import FixedLabelSpaceClusterer
from skmultilearn.ensemble import RakelO
from skmultilearn.model_selection import IterativeStratification
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier

from mlxtend.evaluate.time_series import GroupTimeSeriesSplit
from mlxtend.evaluate.time_series import plot_splits
from mlxtend.evaluate.time_series import print_cv_info
from mlxtend.evaluate.time_series import print_split_info
import optuna
from functools import partial
from collected_functions import *

Set base variable values

In [19]:
random_state = 42
np.random.seed(random_state)


In [20]:
def display_label_cardinality_error(
        y_true: pd.DataFrame,
        y_pred: np.ndarray,
    ) ->None:
    '''
    Create a scatterplot from y_true and y_pred to show accuracy of number of labels predicted.
    Created with the help of Google Gemini
    '''
    if not isinstance(y_pred, np.ndarray):
        y_pred = y_pred.toarray()

    # 1. Calculate Cardinalities (Sum across the label axis, axis=1)
    true_cardinality = np.sum(y_true, axis=1)
    pred_cardinality = np.sum(y_pred, axis=1)

    # 2. Calculate Hamming Loss per sample
    # Hamming loss for sample i = (number of mismatches) / N_LABELS
    mismatches = np.sum(y_true != y_pred, axis=1)
    hamming_loss_Score_ = mismatches / y_true.shape[1]

    # 3. Create the Visualization DataFrame
    df_error_vis = pd.DataFrame({
        'Sample_ID': [f'Sample_{i}' for i in range(y_true.shape[0])],
        'True_Cardinality': true_cardinality,
        'Predicted_Cardinality': pred_cardinality,
        'Hamming_Loss_Score_': hamming_loss_Score_,
        'Total_Errors': mismatches
    })

    fig = px.scatter(
        df_error_vis,
        x='True_Cardinality',
        y='Predicted_Cardinality',
        # Color the points by the error metric
        color='Hamming_Loss_Score_',
        # Scale the size of the points by the number of errors
        # size='Total_Errors',
        hover_data=['Sample_ID', 'Total_Errors'],
        title='Multilabel Error Analysis: Predicted vs. True Label Cardinality',
        color_continuous_scale=px.colors.sequential.Plasma, # Use a good sequential color scale
    )

    # Set axis limits and labels for a clear square comparison
    max_cardinality = y_true.shape[1]
    fig.update_xaxes(
        range=[-0.5, max_cardinality + 0.5],
        tickvals=np.arange(0, max_cardinality + 1),
        title="True Label Cardinality (Complexity)"
    )
    fig.update_yaxes(
        range=[-0.5, max_cardinality + 0.5],
        tickvals=np.arange(0, max_cardinality + 1),
        title="Predicted Label Cardinality (Model Output)"
    )

    # Add a diagonal line for perfect cardinality prediction (where X=Y)
    fig.add_shape(
        type="line", line=dict(dash='dash', color='gray'),
        x0=0, y0=0, x1=max_cardinality, y1=max_cardinality
    )

    fig.show()

Adjust Pandas settings to display more information before truncation.

In [21]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

Load combined MA and DDS training data

In [22]:
df = pd.read_parquet('combined_training_set_nontime.parquet')
print(df.shape)
df.head()

(6765, 21)


,incidentId,state,incidentTypeMA,region,maType,maPriority,supportFunction,agencyId,maId,declarationType,assistanceRequested,statementOfWork,femaDeclarationString,incidentType,incidentBeginDate,fipsStateCode,designatedIncidentTypes,declarationTitle,year,month,day
0,2024081901,CT,Severe Storm,1,FOS,High,5.0,DOI-USGS,3612EMCTDOI-USGS01,EM,USGS Field measurements of flood-water heights...,"As directed by and in coordination with FEMA, ...",EM-3612-CT,Severe Storm,2024-08-18 00:00:00+00:00,9.0,"Mud/Landslide,Severe Storm,Flood","SEVERE STORMS, FLOODING, LANDSLIDES, AND MUDSL...",2024.0,8.0,18.0
1,2024072801,FL,Tropical Storm,4,FOS,Normal,8.0,VA,4806DRFLVA02,DR,Activate VHA OEM to NRCC ESF-8 PHMS. This is a...,As directed by and in coordination with FEMA a...,DR-4806-FL,Tropical Storm,2024-08-01 00:00:00+00:00,12.0,"Tropical Storm,Hurricane",HURRICANE DEBBY,2024.0,8.0,1.0
2,2024072801,FL,Tropical Storm,4,FOS,Normal,0.0,VA,4806DRFLVA01,DR,Activate VA to NRCC. This is a re-issuance of ...,"As directed by and in coordination with FEMA, ...",DR-4806-FL,Tropical Storm,2024-08-01 00:00:00+00:00,12.0,"Tropical Storm,Hurricane",HURRICANE DEBBY,2024.0,8.0,1.0
3,2024072801,FL,Tropical Storm,4,FOS,Normal,4.0,USDA-FS,4806DRFLUSDA-FS01,DR,Activate USFS to the NRCC. This is a re-issuan...,"As directed by and in coordination with FEMA, ...",DR-4806-FL,Tropical Storm,2024-08-01 00:00:00+00:00,12.0,"Tropical Storm,Hurricane",HURRICANE DEBBY,2024.0,8.0,1.0
4,2024072801,FL,Tropical Storm,4,FOS,Normal,11.0,USDA-APH,4806DRFLUSDA-APH01,DR,"USDA liaison(s) to the NRCC, FEMA teams, or ot...","As directed by and in coordination with FEMA, ...",DR-4806-FL,Tropical Storm,2024-08-01 00:00:00+00:00,12.0,"Tropical Storm,Hurricane",HURRICANE DEBBY,2024.0,8.0,1.0


Transform the training data set using approach that is suitable for multiple categorical feature columns and a multilabel target. We return the transformers so that the test data set can be transformed later.

In [23]:
feature_columns = [
    'incidentId',
    'state',
    'designatedIncidentTypes',
    'declarationType',
    'region',
    'year',
]

target_column = 'supportFunction'

X_train, y_train = create_set(df, feature_columns, target_column)

print(X_train.shape, len(y_train))
print(y_train[:5])
X_train.head(5)

(464, 6) 464
0                         [7.0]
1                         [7.0]
2                         [7.0]
3    [2.0, 3.0, 7.0, 9.0, 13.0]
4        [8.0, 1.0, 10.0, 13.0]
Name: supportFunction, dtype: object


,incidentId,state,designatedIncidentTypes,declarationType,region,year
0,2013020801,CT,Severe Storm,DR,1,2013.0
1,2013020801,NH,Severe Storm,DR,1,2013.0
2,2013020801,RI,Severe Storm,DR,1,2013.0
3,2013041203,ND,Flood,EM,8,2013.0
4,2013041501,MA,Terrorist,EM,1,2013.0


In [24]:
# Split the string in designatedIncidentTypes to a list of strings.
# This cannot be done earlier due to groupby issues.
X_train['designatedIncidentTypes'] = X_train['designatedIncidentTypes'].str.split(',')


In [25]:
# Since designatedIncidentTypes is a list it must be transformed differently than the other
# feature columns.
feature_columns_transform = [
    'state',
    'declarationType',
    'region',
]

X_train_transformed, y_train_transformed, preprocessor_X, preprocessor_y = transform_x_y_train(
    X_train,
    y_train,
    feature_columns_transform,
    ['designatedIncidentTypes'],
    target_column
)

In [26]:
# Assign the year values to a variable, then drop year and incidentId from X_train_transformed.
groups_by_year = X_train_transformed['year'].values
X_train_transformed.drop(columns=['incidentId', 'year'], inplace=True)
print(X_train_transformed.shape)
X_train_transformed.head()

(464, 89)


,state_AK,state_AL,state_AR,state_AS,state_AZ,state_CA,state_CO,state_CT,state_DC,state_DE,state_FL,state_GA,state_GU,state_HI,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_MP,state_MS,state_MT,state_NC,state_ND,state_NE,state_NH,state_NJ,state_NM,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_PR,state_RI,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_VI,state_VT,state_WA,state_WI,state_WV,state_WY,declarationType_DR,declarationType_EM,region_1,region_2,region_3,region_4,region_5,region_6,region_7,region_8,region_9,region_10,designatedIncidentTypes_Biological,designatedIncidentTypes_Chemical,designatedIncidentTypes_Coastal Storm,designatedIncidentTypes_Dam/Levee Break,designatedIncidentTypes_Earthquake,designatedIncidentTypes_Fire,designatedIncidentTypes_Flood,designatedIncidentTypes_Hurricane,designatedIncidentTypes_Mud/Landslide,designatedIncidentTypes_Other,designatedIncidentTypes_Severe Ice Storm,designatedIncidentTypes_Severe Storm,designatedIncidentTypes_Snowstorm,designatedIncidentTypes_Straight-Line Winds,designatedIncidentTypes_Terrorist,designatedIncidentTypes_Tornado,designatedIncidentTypes_Tropical Depression,designatedIncidentTypes_Tropical Storm,designatedIncidentTypes_Typhoon,designatedIncidentTypes_Volcanic Eruption,designatedIncidentTypes_Winter Storm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
print(y_train_transformed.shape)
y_train_transformed.head()

(464, 16)


,ESF_0,ESF_1,ESF_2,ESF_3,ESF_4,ESF_5,ESF_6,ESF_7,ESF_8,ESF_9,ESF_10,ESF_11,ESF_12,ESF_13,ESF_14,ESF_15
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0


Run a variety of machine learning models and base classifiers using cross validation for model selection purposes.

In [35]:
base_svc = SVC(
        gamma='auto',
        decision_function_shape='ovo',
        class_weight='balanced',
        random_state=random_state,
        )
base_xgb = XGBClassifier(
        max_depth=12,
        random_state=0,
        n_estimators=100,
        base_score=0.5,
        )
base_gnb = GaussianNB()

br_classifier_svc = BinaryRelevance(
    classifier = base_svc,
    require_dense = [True, True]
)
br_classifier_xgb = BinaryRelevance(
    classifier = base_xgb,
    require_dense = [True, True]
)
br_classifier_gnb = BinaryRelevance(
    classifier = base_gnb,
    require_dense = [True, True]
)

cc_classifier_svc = ClassifierChain(
    classifier=base_svc, 
    require_dense=[True, True],
    order = list(range(0,16)),
)
cc_classifier_xgb = ClassifierChain(
    classifier=base_xgb,
    require_dense=[True, True],
    order = list(range(0,16)),
)
cc_classifier_gnb = ClassifierChain(
    classifier=base_gnb,
    require_dense=[True, True],
    order = list(range(0,16)),
)

classifier_rake_svc = RakelO(
   base_classifier=base_svc,
    base_classifier_require_dense=[True, True],
    labelset_size=3,
    model_count=32
)
classifier_rake_xgb = RakelO(
   base_classifier=base_xgb,
    base_classifier_require_dense=[True, True],
    labelset_size=3,
    model_count=32
)
classifier_rake_gnb = RakelO(
   base_classifier=base_gnb,
    base_classifier_require_dense=[True, True],
    labelset_size=3,
    model_count=32
)

moc_svc = MultiOutputClassifier(estimator=base_svc)
moc_xgb = MultiOutputClassifier(estimator=base_xgb)
moc_gnb = MultiOutputClassifier(estimator=base_gnb)

ovr_svc = OneVsRestClassifier(estimator=base_svc)
ovr_xgb = OneVsRestClassifier(estimator=base_xgb)
ovr_gnb = OneVsRestClassifier(estimator=base_gnb)

clf_list = [
    br_classifier_svc,
    br_classifier_xgb,
    br_classifier_gnb,
    cc_classifier_svc,
    cc_classifier_xgb,
    cc_classifier_gnb,
    classifier_rake_svc,
    classifier_rake_xgb,
    classifier_rake_gnb,
    moc_svc,
    moc_xgb,
    moc_gnb,
    ovr_svc,
    ovr_xgb,
    ovr_gnb,
]

hamming_scorer = make_scorer(
    hamming_loss,
    greater_is_better=False, 
)

F1_Macro_Avg = make_scorer(
    f1_score,
    average='macro', 
    zero_division=1,
)

F1_Samples_Avg = make_scorer(
    f1_score,
    average='macro', 
    zero_division=1,
)

scoring = {
    'Hamming':hamming_scorer,
    'F1_Macro_Avg':F1_Macro_Avg,
    'F1_Samples_Avg':F1_Samples_Avg,
    'jaccard_samples':'jaccard_samples',
}
import warnings
warnings.filterwarnings(
    "ignore", 
    category=UserWarning, 
    module="sklearn.utils.validation" 
)

for clf in clf_list:
    cross_val_results = cross_validate(clf,
                                       X_train_transformed,
                                       y_train_transformed,
                                       cv=10,
                                       scoring=scoring)

    print(f'{clf} Mean Hamming: {cross_val_results['test_Hamming'].mean()}, {cross_val_results['test_Hamming'].std() }')
    print(f'\nMean F1 Macro Avg: {cross_val_results['test_F1_Macro_Avg'].mean()}, {cross_val_results['test_F1_Macro_Avg'].std() }')
    print(f'\nMean F1 Samples Avg: {cross_val_results['test_F1_Samples_Avg'].mean()}, {cross_val_results['test_F1_Samples_Avg'].std() }')
    print(f'\nMean Jaccard Samples: {cross_val_results['test_jaccard_samples'].mean()}, {cross_val_results['test_jaccard_samples'].std() }')

BinaryRelevance(classifier=SVC(class_weight='balanced',
                               decision_function_shape='ovo', gamma='auto',
                               random_state=42),
                require_dense=[True, True]) Mean Hamming: -0.28307990286771506, 0.053525765730321806

Mean F1 Macro Avg: 0.4548263375898728, 0.08349484294504565

Mean F1 Samples Avg: 0.4548263375898728, 0.08349484294504565

Mean Jaccard Samples: 0.4150472199709017, 0.033436553969198306
BinaryRelevance(classifier=XGBClassifier(base_score=0.5, booster=None,
                                         callbacks=None, colsample_bylevel=None,
                                         colsample_bynode=None,
                                         colsample_bytree=None, device=None,
                                         early_stopping_rounds=None,
                                         enable_categorical=False,
                                         eval_metric=None, feature_types=None,
                        

XGBoost appears to be our best base model. The most promising models appear to be BinaryRelevance, ClassifierChain, and OneVsRestClassifier (MultiOutputClassification had similar results and OneVsRestClassifier is more suited to this sort of task per documentation).

The RakelO, though it did well, is being excluded due to its lack of predict_proba method.

We can start with hyperparameter tuning of BinaryRelevance and the XGBoost base classifier.

In [36]:
def objective_brxgb(trial, X, y, groups, cv_args):

    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 7, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 3, 10, log=True),
        'base_score': 0.5, 
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0,
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }

    
    base_XGB = XGBClassifier(**xgb_params)
    
    # The RakelO multilabel classifier (the main estimator)
    clf = BinaryRelevance(
        classifier = base_XGB,
        require_dense = [True, True]
    )

    scores = []    
    # cv = GroupTimeSeriesSplit(**cv_args)
    cv = KFold(**cv_args)

    # for train_index, val_index in cv.split(X, y, groups=groups):
    for train_index, val_index in cv.split(X, y):
    
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        
        # hl = hamming_loss(y_val, y_pred)
        # hl = f1_score(y_val, y_pred, average='samples', zero_division=1)
        eval_scorer = jaccard_score(y_val, y_pred, average='samples')

        scores.append(eval_scorer)

    return np.mean(scores)

cv_args = {'n_splits': 5, 'shuffle': True, 'random_state': 42}

objective_brxgb = partial(objective_brxgb,
                        X=X_train_transformed,
                        y=y_train_transformed,
                        groups=groups_by_year,
                        cv_args=cv_args,
                        )

# 3. Create and Run the Optuna Study
study_brxgb = optuna.create_study(direction="maximize") # Maximize the F1 score

study_brxgb.optimize(objective_brxgb, n_trials=30, show_progress_bar=True)

# --- Final Results ---
print("Optuna Hyperparameter Tuning Complete")
print(f"Best Score (Validation): {study_brxgb.best_value:.4f}")
print("Best Parameters:")
for key, value in study_brxgb.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 11:56:21,120] A new study created in memory with name: no-name-0d465e63-5754-46fa-a13e-f9c4d47f2df9
Best trial: 0. Best value: 0.436215:   3%|▎         | 1/30 [00:14<07:12, 14.93s/it]

[I 2025-11-15 11:56:36,086] Trial 0 finished with value: 0.43621520828919147 and parameters: {'n_estimators': 125, 'max_depth': 11, 'learning_rate': 0.008672111956764726, 'subsample': 0.8793320916019878, 'colsample_bytree': 0.6370676612002565, 'scale_pos_weight': 6.105870414223613}. Best is trial 0 with value: 0.43621520828919147.


Best trial: 0. Best value: 0.436215:   7%|▋         | 2/30 [00:28<06:34, 14.08s/it]

[I 2025-11-15 11:56:49,573] Trial 1 finished with value: 0.4340196822034129 and parameters: {'n_estimators': 200, 'max_depth': 12, 'learning_rate': 0.0020823030389150086, 'subsample': 0.7291066072338501, 'colsample_bytree': 0.9190603510700135, 'scale_pos_weight': 7.6439047625145236}. Best is trial 0 with value: 0.43621520828919147.


Best trial: 0. Best value: 0.436215:  10%|█         | 3/30 [00:40<05:51, 13.00s/it]

...

[I 2025-11-15 12:01:03,218] Trial 27 finished with value: 0.4464461169317411 and parameters: {'n_estimators': 175, 'max_depth': 10, 'learning_rate': 0.0019382912568844273, 'subsample': 0.6981806822947625, 'colsample_bytree': 0.6035989353988976, 'scale_pos_weight': 4.2231701098074375}. Best is trial 23 with value: 0.4529795664052326.


Best trial: 23. Best value: 0.45298:  97%|█████████▋| 29/30 [04:48<00:09,  9.13s/it]

[I 2025-11-15 12:01:09,595] Trial 28 finished with value: 0.4475535449693934 and parameters: {'n_estimators': 100, 'max_depth': 14, 'learning_rate': 0.001010330238093055, 'subsample': 0.6997445406897795, 'colsample_bytree': 0.7594323698206419, 'scale_pos_weight': 4.880507893112039}. Best is trial 23 with value: 0.4529795664052326.


Best trial: 23. Best value: 0.45298: 100%|██████████| 30/30 [04:56<00:00,  9.88s/it]

[I 2025-11-15 12:01:17,469] Trial 29 finished with value: 0.44494517644061826 and parameters: {'n_estimators': 150, 'max_depth': 11, 'learning_rate': 0.00978883946025105, 'subsample': 0.6293554972973382, 'colsample_bytree': 0.647197499998092, 'scale_pos_weight': 6.334117400805296}. Best is trial 23 with value: 0.4529795664052326.
Optuna Hyperparameter Tuning Complete
Best Score (Validation): 0.4530
Best Parameters:
  n_estimators: 125
  max_depth: 13
  learning_rate: 0.0017551798895824016
  subsample: 0.6401440806160666
  colsample_bytree: 0.6688157510746193
  scale_pos_weight: 3.3564999155982322


In [37]:
study_brxgb_best_params = study_brxgb.best_params

study_brxgb_best_params_xgb = {
    'n_estimators': study_brxgb_best_params['n_estimators'],
    'max_depth': study_brxgb_best_params['max_depth'],
    'learning_rate': study_brxgb_best_params['learning_rate'],
    'subsample': study_brxgb_best_params['subsample'],
    'colsample_bytree' : study_brxgb_best_params['colsample_bytree'],
    'scale_pos_weight': study_brxgb_best_params['scale_pos_weight'],
    'base_score': 0.5,
    'random_state': 42,
    'verbosity': 0,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

study_brxgb_base_xgb = XGBClassifier(**study_brxgb_best_params_xgb)
study_brxgb_tuned = BinaryRelevance(
    classifier = study_brxgb_base_xgb,
    require_dense = [True, True]
)

study_brxgb_tuned.fit(X_train_transformed, y_train_transformed)

y_pred_training_br = study_brxgb_tuned.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_br.toarray())

display_error_heatmap(y_train_transformed, y_pred_training_br)

display_label_cardinality_error(y_train_transformed, y_pred_training_br.toarray())

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      0      0      0      0      0      0      1      0      0   
1      0      0      0      0      0      0      0      1      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      1      0      0      0      1      0      1   
4      0      1      0      0      0      0      0      0      1      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       0       0       0       0       0       0  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       1       0       0  
4       1       0       0       1       0       0  

Predicted Multi-Label Targets:
[[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [1 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0]
 [0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



We see some rows with many false positives in the error heatmap, and we see that many of our predictions include too many labels and others too few.

The ClassifierChain we started with used a base order for the labels. We can try to improve the outcome using correlation-based ordering.

In [38]:
# 1. Ensure the correlation matrix has integer indices/columns
if isinstance(y_train_transformed, np.ndarray):
    # If starting from NumPy, force integer columns
    Y_df = pd.DataFrame(y_train_transformed, columns=list(range(y_train_transformed.shape[1])))
else:
    # If starting from a DataFrame, reset the columns to integers
    Y_df = y_train_transformed.copy()
    Y_df.columns = list(range(Y_df.shape[1]))
    
correlation_matrix = Y_df.corr() 

num_labels = correlation_matrix.shape[0]
label_indices = list(range(num_labels))

In [39]:
total_correlations = correlation_matrix.abs().sum(axis=1)

start_label = total_correlations.idxmax()

ordered_labels = [start_label]
remaining_labels = set(label_indices) - {start_label}

In [40]:
while remaining_labels:
    best_next_label = -1
    max_avg_corr = -1

    for candidate_label in remaining_labels:
        # Calculate the absolute correlation between the candidate and all chosen labels
        corrs_with_chosen = correlation_matrix.loc[candidate_label, ordered_labels].abs()
        
        # Calculate the average (or sum) of these correlations
        avg_corr = corrs_with_chosen.mean() 

        if avg_corr > max_avg_corr:
            max_avg_corr = avg_corr
            best_next_label = candidate_label

    # Add the best label to the order and remove it from the remaining set
    if best_next_label != -1:
        ordered_labels.append(best_next_label)
        remaining_labels.remove(best_next_label)
    else:
        # Should not happen if the correlation matrix is correctly defined
        break

# The final result is the desired order for the ClassifierChain
final_chain_order = ordered_labels
print(f"Optimal ClassifierChain Order: {final_chain_order}")

Optimal ClassifierChain Order: [np.int64(1), 12, 10, 11, 2, 4, 6, 5, 8, 9, 13, 3, 0, 14, 15, 7]


In [41]:
def objective_ccxgb(trial, X, y, groups, cv_args, final_chain_order):

    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 7, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 3, 10, log=True),
        'base_score': 0.5, 
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0,
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }
        
    base_XGB = XGBClassifier(**xgb_params)
    
    # The RakelO multilabel classifier (the main estimator)
    clf = ClassifierChain(
        classifier = base_XGB,
        require_dense = [True, True],
        order = final_chain_order,
    )

    scores = []    
    # cv = GroupTimeSeriesSplit(**cv_args)
    cv = KFold(**cv_args)

    # for train_index, val_index in cv.split(X, y, groups=groups):
    for train_index, val_index in cv.split(X, y):
    
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        
        # hl = hamming_loss(y_val, y_pred)
        # hl = f1_score(y_val, y_pred, average='samples', zero_division=1)
        eval_scorer = jaccard_score(y_val, y_pred, average='samples')

        scores.append(eval_scorer)

    return np.mean(scores)

cv_args = {'n_splits': 5, 'shuffle': True, 'random_state': 42}

objective_ccxgb = partial(objective_ccxgb,
                        X=X_train_transformed,
                        y=y_train_transformed,
                        groups=groups_by_year,
                        cv_args=cv_args,
                        final_chain_order=final_chain_order
                        )

# 3. Create and Run the Optuna Study
study_ccxgb = optuna.create_study(direction="maximize") # Maximize the F1 score

study_ccxgb.optimize(objective_ccxgb, n_trials=30, show_progress_bar=True)

# --- Final Results ---
print("Optuna Hyperparameter Tuning Complete")
print(f"Best Score (Validation): {study_ccxgb.best_value:.4f}")
print("Best Parameters:")
for key, value in study_ccxgb.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 12:01:21,340] A new study created in memory with name: no-name-604c9a9e-3efa-49c9-b2ca-3fd354b889bb
Best trial: 0. Best value: 0.206292:   3%|▎         | 1/30 [00:05<02:29,  5.16s/it]

[I 2025-11-15 12:01:26,507] Trial 0 finished with value: 0.20629191862887514 and parameters: {'n_estimators': 75, 'max_depth': 14, 'learning_rate': 0.03237871429317957, 'subsample': 0.7675029662130528, 'colsample_bytree': 0.6187989170536127, 'scale_pos_weight': 6.404975529761505}. Best is trial 0 with value: 0.20629191862887514.


Best trial: 0. Best value: 0.206292:   7%|▋         | 2/30 [00:16<04:01,  8.64s/it]

[I 2025-11-15 12:01:37,577] Trial 1 finished with value: 0.19810747979489213 and parameters: {'n_estimators': 200, 'max_depth': 11, 'learning_rate': 0.00490403357936558, 'subsample': 0.7389507844302405, 'colsample_bytree': 0.8895901079909583, 'scale_pos_weight': 5.766488889587937}. Best is trial 0 with value: 0.20629191862887514.


Best trial: 2. Best value: 0.214182:  10%|█         | 3/30 [00:23<03:32,  7.88s/it]

...

[I 2025-11-15 12:05:01,790] Trial 27 finished with value: 0.2269031973818089 and parameters: {'n_estimators': 150, 'max_depth': 10, 'learning_rate': 0.00867910650796973, 'subsample': 0.9050290964288319, 'colsample_bytree': 0.7308057550290843, 'scale_pos_weight': 9.744756345423458}. Best is trial 25 with value: 0.23287634132918145.


Best trial: 25. Best value: 0.232876:  97%|█████████▋| 29/30 [03:54<00:11, 11.46s/it]

[I 2025-11-15 12:05:15,702] Trial 28 finished with value: 0.22776752187867197 and parameters: {'n_estimators': 175, 'max_depth': 10, 'learning_rate': 0.007078747914075915, 'subsample': 0.8965521895686752, 'colsample_bytree': 0.7333408789640687, 'scale_pos_weight': 9.93002541230022}. Best is trial 25 with value: 0.23287634132918145.


Best trial: 25. Best value: 0.232876: 100%|██████████| 30/30 [04:03<00:00,  8.13s/it]

[I 2025-11-15 12:05:25,216] Trial 29 finished with value: 0.22738182853715813 and parameters: {'n_estimators': 175, 'max_depth': 8, 'learning_rate': 0.00377030598582794, 'subsample': 0.787459604585502, 'colsample_bytree': 0.7500128894815262, 'scale_pos_weight': 8.877333862952788}. Best is trial 25 with value: 0.23287634132918145.
Optuna Hyperparameter Tuning Complete
Best Score (Validation): 0.2329
Best Parameters:
  n_estimators: 150
  max_depth: 10
  learning_rate: 0.004040644168123389
  subsample: 0.8946510229960065
  colsample_bytree: 0.7150608320012486
  scale_pos_weight: 9.847087176673588


In [42]:
study_ccxgb_best_params = study_ccxgb.best_params

study_ccxgb_best_params_xgb = {
    'n_estimators': study_ccxgb_best_params['n_estimators'],
    'max_depth': study_ccxgb_best_params['max_depth'],
    'learning_rate': study_ccxgb_best_params['learning_rate'],
    'subsample': study_ccxgb_best_params['subsample'],
    'colsample_bytree' : study_ccxgb_best_params['colsample_bytree'],
    'scale_pos_weight': study_ccxgb_best_params['scale_pos_weight'],
    'base_score': 0.5,
    'random_state': 42,
    'verbosity': 0,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

study_ccxgb_bbase_xgb = XGBClassifier(**study_ccxgb_best_params_xgb)
study_ccxgb_tuned = ClassifierChain(
    classifier = study_ccxgb_bbase_xgb,
    require_dense = [True, True],
    order=final_chain_order,
)

study_ccxgb_tuned.fit(X_train_transformed, y_train_transformed)

y_pred_training_cc = study_ccxgb_tuned.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_cc.toarray())

display_error_heatmap(y_train_transformed, y_pred_training_cc.toarray())

display_label_cardinality_error(y_train_transformed, y_pred_training_cc.toarray())

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      0      0      0      0      0      0      1      0      0   
1      0      0      0      0      0      0      0      1      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      1      0      0      0      1      0      1   
4      0      1      0      0      0      0      0      0      1      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       0       0       0       0       0       0  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       1       0       0  
4       1       0       0       1       0       0  

Predicted Multi-Label Targets:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



It's certainly more aggressive in the number of labels it predicts than the first model, whic explains the high number of false positives in the error heatmap. We can try one instance using the standard order for comparison.

In [43]:
def objective_ccxgb2(trial, X, y, groups, cv_args, final_chain_order):

    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 7, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 3, 10, log=True),
        'base_score': 0.5, 
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0,
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }
        
    base_XGB = XGBClassifier(**xgb_params)
    
    # The RakelO multilabel classifier (the main estimator)
    clf = ClassifierChain(
        classifier = base_XGB,
        require_dense = [True, True],
        order = final_chain_order,
    )

    scores = []    
    # cv = GroupTimeSeriesSplit(**cv_args)
    cv = KFold(**cv_args)

    # for train_index, val_index in cv.split(X, y, groups=groups):
    for train_index, val_index in cv.split(X, y):
    
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        
        # hl = hamming_loss(y_val, y_pred)
        # hl = f1_score(y_val, y_pred, average='samples', zero_division=1)
        eval_scorer = jaccard_score(y_val, y_pred, average='samples')

        scores.append(eval_scorer)

    return np.mean(scores)

cv_args = {'n_splits': 5, 'shuffle': True, 'random_state': 42}

objective_ccxgb2 = partial(objective_ccxgb2,
                        X=X_train_transformed,
                        y=y_train_transformed,
                        groups=groups_by_year,
                        cv_args=cv_args,
                        final_chain_order=list(range(0,16)),
                        )

# 3. Create and Run the Optuna Study
study_ccxgb2 = optuna.create_study(direction="maximize") # Maximize the F1 score

study_ccxgb2.optimize(objective_ccxgb2, n_trials=30, show_progress_bar=True)

# --- Final Results ---
print("Optuna Hyperparameter Tuning Complete")
print(f"Best Score (Validation): {study_ccxgb2.best_value:.4f}")
print("Best Parameters:")
for key, value in study_ccxgb2.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 12:05:27,131] A new study created in memory with name: no-name-ce528762-fa34-4a20-8eca-2c979a7a3739
Best trial: 0. Best value: 0.43685:   3%|▎         | 1/30 [00:09<04:49,  9.98s/it]

[I 2025-11-15 12:05:37,112] Trial 0 finished with value: 0.43684996779491875 and parameters: {'n_estimators': 150, 'max_depth': 14, 'learning_rate': 0.015457729520398762, 'subsample': 0.9039915584276619, 'colsample_bytree': 0.6030342250019809, 'scale_pos_weight': 6.352671686807695}. Best is trial 0 with value: 0.43684996779491875.


Best trial: 1. Best value: 0.43746:   7%|▋         | 2/30 [00:13<02:51,  6.12s/it]

[I 2025-11-15 12:05:40,531] Trial 1 finished with value: 0.43746029320012497 and parameters: {'n_estimators': 75, 'max_depth': 8, 'learning_rate': 0.09604893583068523, 'subsample': 0.8253307995570496, 'colsample_bytree': 0.8895043652207858, 'scale_pos_weight': 3.5614909135532162}. Best is trial 1 with value: 0.43746029320012497.


Best trial: 1. Best value: 0.43746:  10%|█         | 3/30 [00:16<02:08,  4.74s/it]

...

[I 2025-11-15 12:08:48,323] Trial 27 finished with value: 0.45302499559424525 and parameters: {'n_estimators': 75, 'max_depth': 14, 'learning_rate': 0.014106549802934546, 'subsample': 0.6861763262424767, 'colsample_bytree': 0.6122491316315477, 'scale_pos_weight': 4.192506530582557}. Best is trial 23 with value: 0.4611951914897217.


Best trial: 23. Best value: 0.461195:  97%|█████████▋| 29/30 [03:26<00:06,  6.18s/it]

[I 2025-11-15 12:08:54,060] Trial 28 finished with value: 0.45797443154736284 and parameters: {'n_estimators': 125, 'max_depth': 10, 'learning_rate': 0.024264760489518488, 'subsample': 0.6646072885035874, 'colsample_bytree': 0.6517591040123977, 'scale_pos_weight': 3.8189996212229054}. Best is trial 23 with value: 0.4611951914897217.


Best trial: 23. Best value: 0.461195: 100%|██████████| 30/30 [03:34<00:00,  7.15s/it]

[I 2025-11-15 12:09:01,728] Trial 29 finished with value: 0.4577663187936681 and parameters: {'n_estimators': 150, 'max_depth': 14, 'learning_rate': 0.018431599277604373, 'subsample': 0.7426958271741206, 'colsample_bytree': 0.6153675155283195, 'scale_pos_weight': 3.2730543762813604}. Best is trial 23 with value: 0.4611951914897217.
Optuna Hyperparameter Tuning Complete
Best Score (Validation): 0.4612
Best Parameters:
  n_estimators: 125
  max_depth: 14
  learning_rate: 0.010947959418489094
  subsample: 0.7510917361183614
  colsample_bytree: 0.7054987571953771
  scale_pos_weight: 3.2172303041415438


In [44]:
study_ccxgb2_best_params = study_ccxgb2.best_params

study_ccxgb2_best_params_xgb = {
    'n_estimators': study_ccxgb2_best_params['n_estimators'],
    'max_depth': study_ccxgb2_best_params['max_depth'],
    'learning_rate': study_ccxgb2_best_params['learning_rate'],
    'subsample': study_ccxgb2_best_params['subsample'],
    'colsample_bytree' : study_ccxgb2_best_params['colsample_bytree'],
    'scale_pos_weight': study_ccxgb2_best_params['scale_pos_weight'],
    'base_score': 0.5,
    'random_state': 42,
    'verbosity': 0,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

study_ccxgb2_bbase_xgb = XGBClassifier(**study_ccxgb2_best_params_xgb)
study_ccxgb2_tuned = ClassifierChain(
    classifier = study_ccxgb2_bbase_xgb,
    require_dense = [True, True],
    order=list(range(0,16)),
)

study_ccxgb2_tuned.fit(X_train_transformed, y_train_transformed)

y_pred_training_cc2 = study_ccxgb2_tuned.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_cc2.toarray())

display_error_heatmap(y_train_transformed, y_pred_training_cc2.toarray())

display_label_cardinality_error(y_train_transformed, y_pred_training_cc2.toarray())

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      0      0      0      0      0      0      1      0      0   
1      0      0      0      0      0      0      0      1      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      1      0      0      0      1      0      1   
4      0      1      0      0      0      0      0      0      1      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       0       0       0       0       0       0  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       1       0       0  
4       1       0       0       1       0       0  

Predicted Multi-Label Targets:
[[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



The metrics are improved but the model is still aggressive in predicting false positives. Our final hypertuning instance will be for OneVsRest.

In [45]:
def objective_ovrxgb(trial, X, y, groups, cv_args):

    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 7, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 3, 10, log=True),
        'base_score': 0.5, 
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0,
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }
        
    base_XGB = XGBClassifier(**xgb_params)
    
    # The RakelO multilabel classifier (the main estimator)
    clf = OneVsRestClassifier(
        estimator = base_XGB,
    )

    scores = []    
    # cv = GroupTimeSeriesSplit(**cv_args)
    cv = KFold(**cv_args)

    # for train_index, val_index in cv.split(X, y, groups=groups):
    for train_index, val_index in cv.split(X, y):
    
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]
        
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        
        # hl = hamming_loss(y_val, y_pred)
        # hl = f1_score(y_val, y_pred, average='samples', zero_division=1)
        eval_scorer = jaccard_score(y_val, y_pred, average='samples')

        scores.append(eval_scorer)

    return np.mean(scores)

cv_args = {'n_splits': 5, 'shuffle': True, 'random_state': 42}

objective_ovrxgb = partial(objective_ovrxgb,
                        X=X_train_transformed,
                        y=y_train_transformed,
                        groups=groups_by_year,
                        cv_args=cv_args,
                        )

# 3. Create and Run the Optuna Study
study_ovrxgb = optuna.create_study(direction="maximize") # Maximize the F1 score

study_ovrxgb.optimize(objective_ovrxgb, n_trials=30, show_progress_bar=True)

# --- Final Results ---
print("Optuna Hyperparameter Tuning Complete")
print(f"Best Score (Validation): {study_ovrxgb.best_value:.4f}")
print("Best Parameters:")
for key, value in study_ovrxgb.best_params.items():
    print(f"  {key}: {value}")

[I 2025-11-15 12:09:03,464] A new study created in memory with name: no-name-35b65f16-0211-4daa-87b0-e77a49f73762
Best trial: 0. Best value: 0.447276:   3%|▎         | 1/30 [00:08<04:13,  8.75s/it]

[I 2025-11-15 12:09:12,209] Trial 0 finished with value: 0.44727640171889826 and parameters: {'n_estimators': 100, 'max_depth': 13, 'learning_rate': 0.05315011823375804, 'subsample': 0.7195584157670519, 'colsample_bytree': 0.7430524742003806, 'scale_pos_weight': 3.1498100004565583}. Best is trial 0 with value: 0.44727640171889826.


Best trial: 0. Best value: 0.447276:   7%|▋         | 2/30 [00:17<04:11,  8.98s/it]

[I 2025-11-15 12:09:21,354] Trial 1 finished with value: 0.43904932394238144 and parameters: {'n_estimators': 125, 'max_depth': 11, 'learning_rate': 0.016161120201734963, 'subsample': 0.917193066760267, 'colsample_bytree': 0.6708213089913093, 'scale_pos_weight': 5.4899726676240315}. Best is trial 0 with value: 0.44727640171889826.


Best trial: 2. Best value: 0.451127:  10%|█         | 3/30 [00:29<04:29,  9.97s/it]

...

[I 2025-11-15 12:14:37,773] Trial 27 finished with value: 0.44784055018276614 and parameters: {'n_estimators': 150, 'max_depth': 12, 'learning_rate': 0.006991336666430261, 'subsample': 0.7444077620672404, 'colsample_bytree': 0.6269755346459708, 'scale_pos_weight': 3.771432337948326}. Best is trial 22 with value: 0.46174205060951207.


Best trial: 22. Best value: 0.461742:  97%|█████████▋| 29/30 [05:46<00:11, 11.72s/it]

[I 2025-11-15 12:14:50,224] Trial 28 finished with value: 0.4465442635191934 and parameters: {'n_estimators': 175, 'max_depth': 10, 'learning_rate': 0.06505870128454658, 'subsample': 0.6288279972621109, 'colsample_bytree': 0.6759656667278682, 'scale_pos_weight': 5.038810100749083}. Best is trial 22 with value: 0.46174205060951207.


Best trial: 22. Best value: 0.461742: 100%|██████████| 30/30 [06:01<00:00, 12.04s/it]

[I 2025-11-15 12:15:04,529] Trial 29 finished with value: 0.4545116028198076 and parameters: {'n_estimators': 125, 'max_depth': 7, 'learning_rate': 0.042117116525461545, 'subsample': 0.670613797142382, 'colsample_bytree': 0.7683065434200231, 'scale_pos_weight': 3.1761210374158555}. Best is trial 22 with value: 0.46174205060951207.
Optuna Hyperparameter Tuning Complete
Best Score (Validation): 0.4617
Best Parameters:
  n_estimators: 125
  max_depth: 7
  learning_rate: 0.030928113120891658
  subsample: 0.6025823757629755
  colsample_bytree: 0.6198233891559969
  scale_pos_weight: 3.421576979607886


In [46]:
study_ovrxgb_best_params = study_ovrxgb.best_params

study_ovrxgb_best_params_xgb = {
    'n_estimators': study_ovrxgb_best_params['n_estimators'],
    'max_depth': study_ovrxgb_best_params['max_depth'],
    'learning_rate': study_ovrxgb_best_params['learning_rate'],
    'subsample': study_ovrxgb_best_params['subsample'],
    'colsample_bytree' : study_ovrxgb_best_params['colsample_bytree'],
    'scale_pos_weight': study_ovrxgb_best_params['scale_pos_weight'],
    'base_score': 0.5,
    'random_state': 42,
    'verbosity': 0,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

study_ovrxgb_bbase_xgb = XGBClassifier(**study_ovrxgb_best_params_xgb)
study_ovrxgb_tuned = OneVsRestClassifier(
    estimator = study_ovrxgb_bbase_xgb,
)

study_ovrxgb_tuned.fit(X_train_transformed, y_train_transformed)

y_pred_training_ovr = study_ovrxgb_tuned.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_ovr)

display_error_heatmap(y_train_transformed, y_pred_training_ovr)

display_label_cardinality_error(y_train_transformed, y_pred_training_ovr)

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      0      0      0      0      0      0      1      0      0   
1      0      0      0      0      0      0      0      1      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      1      0      0      0      1      0      1   
4      0      1      0      0      0      0      0      0      1      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       0       0       0       0       0       0  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       1       0       0  
4       1       0       0       1       0       0  

Predicted Multi-Label Targets:
[[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [1 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0]
 [0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



OneVsRest is still relatively aggressive in terms of false positive but does have the best overall metrics score and an impressive f1_macro score.

We will run one quick check of the untuned classifier for comparison.

In [47]:
ovr_xgb.fit(X_train_transformed, y_train_transformed)

y_pred_training_ovr_base = ovr_xgb.predict(X_train_transformed)

display_results(y_train_transformed, y_pred_training_ovr_base)

display_error_heatmap(y_train_transformed, y_pred_training_ovr_base)

display_label_cardinality_error(y_train_transformed, y_pred_training_ovr_base)

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      0      0      0      0      0      0      1      0      0   
1      0      0      0      0      0      0      0      1      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      1      0      0      0      1      0      1   
4      0      1      0      0      0      0      0      0      1      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       0       0       0       0       0       0  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       1       0       0  
4       1       0       0       1       0       0  

Predicted Multi-Label Targets:
[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



This has fewer false positive but seems more prone to underestimate the number of labels than our tuned model. It even has some instances with no positive label predictions at all, which is not great for our needs.

Since we hope to provide more options for our end user to select, we will move forward with our tuned model even though this has a higher combined metrics score.

We can now load in the test set to measure performance of our tuned model.

In [48]:
df_test = pd.read_parquet('combined_test_set_nontime.parquet')
X_test, y_test = create_set(df_test, feature_columns, target_column)

In [49]:
# Split the string in designatedIncidentTypes to a list of strings.
# This cannot be done earlier due to groupby issues.
X_test['designatedIncidentTypes'] = X_test['designatedIncidentTypes'].str.split(',')
X_test.head()

,incidentId,state,designatedIncidentTypes,declarationType,region,year
0,2012102505,CT,[Hurricane],DR,1,2012.0
1,2012102505,DC,[Hurricane],EM,3,2012.0
2,2012102505,DE,[Hurricane],DR,3,2012.0
3,2012102505,DE,[Hurricane],EM,3,2012.0
4,2012102505,MD,[Hurricane],DR,3,2012.0


In [50]:
y_test.head()

0    [1.0, 3.0, 4.0, 7.0, 8.0, 10.0, 11.0, 12.0, 15.0]
1                                                [4.0]
2                                                [7.0]
3                                                [2.0]
4                                                [7.0]
Name: supportFunction, dtype: object

In [51]:
X_test_transformed, y_test_transformed = transform_x_y_test(
    X_test,
    y_test,
    preprocessor_X,
    preprocessor_y
)
X_test_transformed.drop(columns=['incidentId', 'year'], inplace=True)

# X_test.drop(columns=['incidentId'], inplace=True)

# X_test_transformed = preprocessor_X.transform(X_test)
# feature_names_x = preprocessor_X.get_feature_names_out()
# X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names_x)
# y_test_transformed = preprocessor_y.transform(y_test)
# feature_names_y = preprocessor_y.classes_
# y_test_transformed = pd.DataFrame(y_test_transformed, columns=feature_names_y)


In [52]:
print(X_test_transformed.shape)
X_test_transformed.head()

(101, 89)


,state_AK,state_AL,state_AR,state_AS,state_AZ,state_CA,state_CO,state_CT,state_DC,state_DE,state_FL,state_GA,state_GU,state_HI,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_MP,state_MS,state_MT,state_NC,state_ND,state_NE,state_NH,state_NJ,state_NM,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_PR,state_RI,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_VI,state_VT,state_WA,state_WI,state_WV,state_WY,declarationType_DR,declarationType_EM,region_1,region_2,region_3,region_4,region_5,region_6,region_7,region_8,region_9,region_10,designatedIncidentTypes_Biological,designatedIncidentTypes_Chemical,designatedIncidentTypes_Coastal Storm,designatedIncidentTypes_Dam/Levee Break,designatedIncidentTypes_Earthquake,designatedIncidentTypes_Fire,designatedIncidentTypes_Flood,designatedIncidentTypes_Hurricane,designatedIncidentTypes_Mud/Landslide,designatedIncidentTypes_Other,designatedIncidentTypes_Severe Ice Storm,designatedIncidentTypes_Severe Storm,designatedIncidentTypes_Snowstorm,designatedIncidentTypes_Straight-Line Winds,designatedIncidentTypes_Terrorist,designatedIncidentTypes_Tornado,designatedIncidentTypes_Tropical Depression,designatedIncidentTypes_Tropical Storm,designatedIncidentTypes_Typhoon,designatedIncidentTypes_Volcanic Eruption,designatedIncidentTypes_Winter Storm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
print(y_test_transformed.shape)
y_test_transformed.head()

(101, 16)


,ESF_0,ESF_1,ESF_2,ESF_3,ESF_4,ESF_5,ESF_6,ESF_7,ESF_8,ESF_9,ESF_10,ESF_11,ESF_12,ESF_13,ESF_14,ESF_15
0,0,1,0,1,1,0,0,1,1,0,1,1,1,0,0,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [60]:
y_pred_test_over = study_ovrxgb_tuned.predict(X_test_transformed)

display_results(y_test_transformed, y_pred_test_over)

display_error_heatmap(y_test_transformed, y_pred_test_over)

display_label_cardinality_error(y_test_transformed, y_pred_test_over)



True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      1      0      1      1      0      0      1      1      0   
1      0      0      0      0      1      0      0      0      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      0      0      0      0      0      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       1       1       1       0       0       1  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       0       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0]
 [1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0]
 [1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0]
 [1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



In [57]:
y_proba = study_ovrxgb_tuned.predict_proba(X_test_transformed)

y_proba_adjusted = (y_proba >= 0.33).astype(int)

In [58]:
display_results(y_test_transformed, y_proba_adjusted)

display_error_heatmap(y_test_transformed, y_proba_adjusted)

display_label_cardinality_error(y_test_transformed, y_proba_adjusted)

True Multi-Label Targets:
   ESF_0  ESF_1  ESF_2  ESF_3  ESF_4  ESF_5  ESF_6  ESF_7  ESF_8  ESF_9  \
0      0      1      0      1      1      0      0      1      1      0   
1      0      0      0      0      1      0      0      0      0      0   
2      0      0      0      0      0      0      0      1      0      0   
3      0      0      1      0      0      0      0      0      0      0   
4      0      0      0      0      0      0      0      1      0      0   

   ESF_10  ESF_11  ESF_12  ESF_13  ESF_14  ESF_15  
0       1       1       1       0       0       1  
1       0       0       0       0       0       0  
2       0       0       0       0       0       0  
3       0       0       0       0       0       0  
4       0       0       0       0       0       0  

Predicted Multi-Label Targets:
[[1 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0]
 [1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0]]

Su

/Users/joschw/opt/anaconda3/envs/milestone2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)

